In [32]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint,ode,solve_ivp,OdeSolver,BDF
import scipy.optimize
from array import array
import matplotlib.pyplot as plt


In [33]:
tlist = []
odefluxlist = []
def GI_ode_universal(t, y, p):
    
    # SYSTEM PARAMS
    Eg0 = 0.0118
    k = 0.4861
    BV = 7200
    b = 1553.6

    # Define metabolic rate M and Insulin secretion rate ISR
    Mmax = 1
    alpha_M = 150
    kM = 2
    
    M = Mmax * y[0] ** kM / (alpha_M ** kM + y[0] ** kM)
    
    OGTT_flux = (int(t >0 ) - int(t > p['t1'])) * t * p['a1'] / p['t1'] +\
     (int(t > p['t1']) - int(t > p['t2'])) * ((t - p['t2']) * (p['a2'] - p['a1']) / (p['t2'] - p['t1']) + p['a2']) +\
      (int(t > p['t2']) - int(t > p['t3'])) * (t - p['t3']) * (p['a3'] - p['a2']) / (p['t3'] - p['t2'])

    OGTTbar = 1
    OGTT_rate = OGTTbar * OGTT_flux


    # hepa_IR HGP
    hepa_bar = 15.443
    hepa_k = 0.27
    hepa_b = -3.54277
    hepa_max = hepa_bar / (hepa_k + p['si']) + hepa_b


    alpha_max = 6
    alpha_k = 0.4
    alpha_b = -0.5
    alpha_HGP = alpha_max / (alpha_k + p['si']) + alpha_b


    HGP_b = 0.104166

    HGP = hepa_max / (alpha_HGP + y[1] * p['hepasi']) + HGP_b


    GF_bar = 4.45
    kGF = 16
    alpha_GF = 260
    shGF = -89
    GF_b = 1.78


    GF = (GF_bar * (y[0] - shGF) ** kGF / (alpha_GF ** kGF + (y[0] - shGF) ** kGF)) + GF_b

    ca_bar = 2
    kca = 4
    alpha_ca = 0.62
    ca_b = 0.07

    ci = ca_bar * (M + p['gamma_bar'] * p['gamma']) ** kca / (alpha_ca ** kca + (M + p['gamma_bar'] * p['gamma']) ** kca) + ca_b


    # Define Microdoman Ca2+
    cmd_factor = 150 
    cmd_b = 0.0635
    cik = 4
    cialpha = 1

    cmd = cmd_factor * ci ** cik / (cialpha ** cik + ci ** cik) + cmd_b


    # Define exocytosis
    k1 = 20
    km1 = 100
    r1 = 0.6
    rm1 = 1
    rm2 = 0.001

    r30 = 1.205
    rm3 = 0.0001
    u1 = 2000
    u2 = 3
    u3 = 0.02
    Kp2 = 2.3
    
    ts=60 
    unit_con=0.00069444

    r2 = p['r20'] * ci / (ci + Kp2)
    r3 = p['sigma'] * GF * r30 * ci / (ci + Kp2)
    
    N1_C = km1 / (3 * k1 * cmd + rm1)
    N1_D = r1 / (3 * k1 * cmd + rm1)
    
    N2_E = 3 * k1 * cmd / (2 * k1 * cmd + km1)
    N2_F = 2 * km1 / (2 * k1 * cmd + km1)
    
    N3_L = 2 * k1 * cmd / (2 * km1 + k1 * cmd)
    N3_N = 3 * km1 / (2 * km1 + k1 * cmd)
    
    CN4 = (k1 * cmd / (3 * km1 + u1))
    CN3 = N3_L / (1 - N3_N * CN4)
    CN2 = N2_E / (1 - N2_F * CN3)
    CN1 = N1_D / (1 - N1_C * CN2)
    
    N1 = CN1 * y[2]
    N2 = CN2 * N1
    N3 = CN3 * N2
    N4 = CN4 * N3
    NF = u1 * N4 / u2
    NR = (u2 / u3) * NF
    
    ISR = ts * 9 * (u3 * NR)
    
    dGdt = HGP + OGTT_rate - (Eg0 + unit_con * p['si'] * y[1]) * y[0]
    dIdt = b * ISR / BV - k * y[1]
    
    dN5dt = ts * (rm1 * CN1 * y[2] - (r1 + rm2) * y[2] + r2 * y[3])
    dN6dt = ts * (r3 + rm2 * y[2] - (rm3 + r2) * y[3])
    
    return [dGdt, dIdt, dN5dt, dN6dt]

In [232]:

def OGTT_cost_function_universal(theta0,theta1, data, costparams, odeparams):

    # Ode solver 
    theta= np.array([theta0,theta1])

    tspan = [data2['t'][0], data2['t'][-1]]


    init=costparams['init']
    
    if any(theta<costparams['LB']) or any(theta>costparams['UB']):

        S=1e6
        true_err=1e6
        #return S
        return S, true_err, None, None

    odeparams['sigma']=theta[0]
    odeparams['si']=theta[1]

    odefun = lambda t, y: GI_ode_universal(t, y, odeparams)
    

    so = solve_ivp(odefun, (0, 1400),init,method='BDF',rtol=0.2)

    y = np.array(so.y)
    t = np.array(so.t)


    init=[y[:,-1],t[-1]] 


    sol= solve_ivp(odefun,(0,120), init[0],rtol=0.00001)

    y1 = np.array(sol.y)
    t1 = np.array(sol.t)
    

    
    costparams['doPlot'] = False
    if costparams['doPlot']:
        import matplotlib.pyplot as plt
        plt.plot(data2['t'],data2['G'],'x','MarkerSize',10)
        plt.plot(data2['t'],data2['I'],'X','MarkerSize',10) 
        plt.plot(t1, y1[0,:])
        plt.plot(t1, y1[1,:])
        plt.show()
  
    Gpred = np.interp(data2['t'],t1,y1[0])
    Ipred = np.interp(data2['t'],t1,y1[1])


    OGTTt=[0, 15, 30, 60, 90, 120]

    Gsim=np.interp(OGTTt,t1,y1[0])
    Isim=np.interp(OGTTt,t1,y1[1])


    S=np.sum(costparams['weights_G'].values*(Gpred-data2['G'].values)**2 + costparams['weights_I'].values*(Ipred-data2['I'].values)**2 )
    true_err=np.sum((Gpred-data2['G'])**2 + (Ipred-data2['I'])**2)


    return S, true_err, Gsim, Isim


In [236]:
tdata = [0, 30, 60, 90, 120]
numpts = len(tdata)

data = pd.read_csv('FWS_OGTT_Combined.csv')

outfile = open("WS_fit_bounds.xlsx",'wb')
gdata = data.iloc[:, 8:13]

idata = data.iloc[:,13:14].mean(axis=1).to_frame().join(data.iloc[:,15:19])


odeparams = {
    'a1': 5.99,
    'a2': 2.14,
    'a3': 0.0013,
    't1': 15.60,
    't2': 137.2,
    't3': 258.3,
    'r20': 0.006,
    'hepasi': 1,
    'gamma_bar': 1,
}

# Initial condition for ODE
init = [80.1842, 5.8462, 60.9341, 443.7764]

# Initial condition for fitting

gamma_bar = 1
r20 = 0.006
hepasi = 1

a1 = 8
a2 = 5
a3 = 0

t1 = 15
t2 = 120
t3 = 240

sigma = 1
si = 0.8

theta0 = [sigma, si]

# Parameter bounds
sigmalb = 0.01
sigmaub = 10
silb = 0.01
siub = 3

hepasilb = 0.01
hepasiub = 10
gamma_barlb = -100
gamma_barub = 100

a1lb = 1
a1ub = 20
a2lb = 0.5
a2ub = 10
a3lb = 0
a3ub = 2

t1lb = 5
t1ub = 40
t2lb = 45
t2ub = 200
t3lb = 205
t3ub = 360

r20lb = 0.00006
r20ub = 0.06

LB = [sigmalb, silb]
UB = [sigmaub, siub]

costparams = {
    'init': init,
    'LB': LB,
    'UB': UB,
}

var_G = gdata.std()
var_I = idata.std()

numpar = len(theta0)

results = np.zeros((len(data), 2))
params = np.zeros((len(data), numpar))

In [238]:
for patientID in range(data.shape[0]):
    data2 = {}
    data2['t'] = tdata
    data2['G'] = gdata.iloc[patientID,:]
    data2['I'] = idata.iloc[patientID,:]

    costparams['weights_G'] = 1/var_G
    costparams['weights_I'] = 1/var_I

    missing1 = np.isnan(data2['G']) 
    missing2 = np.isnan(data2['I'])
    missing = np.array(missing1) * np.array(missing1)

    data2['t'] = np.delete(data2['t'], missing)
    data2['G'] = np.delete(data2['G'], missing)
    data2['I'] = np.delete(data2['I'], missing)
    costparams['weights_G'] = np.delete(costparams['weights_G'], missing)
    costparams['weights_I'] = np.delete(costparams['weights_I'], missing)



    odeparams['gamma'] = 2.5e-6*17500

    print(patientID)
    
    def costfun(theta):
        x ,y,z,t = OGTT_cost_function_universal(theta[0],theta[1],data2,costparams,odeparams)
        return x

    res = scipy.optimize.fmin(func = costfun,x0=[1,0.8])
    
    paramin = res

    wrss, true_error, gsim, isim = OGTT_cost_function_universal(paramin[0],paramin[1], data2, costparams, odeparams)
    
    params[patientID,:] = paramin
    
    ss = np.nansum(data2['G']**2) + np.nansum(data2['I']**2)
    rsq = 1 - true_error/ss
    aic = 2*numpar + numpts*np.log(numpts*wrss)
    results[patientID] = [paramin[0],paramin[1]]


results_df = pd.DataFrame(results, columns = ['sigma','si'])
covartab = pd.DataFrame(data[['subject']])

final_df = pd.concat([covartab, results_df], axis=1)
final_df.to_excel(outfile)


0
487.88044239945276
499.0493137828989
516.8818856009104
469.11807738367565
443.8157074179282
434.1608224182117
401.562648150696
354.077808245807
286.10691432999295
251.72620681804844
197.49366540708183
674.4550118379335
310.54983099751104
216.26135292245064
1167.0759938481128
224.59816679108118
317.7170374964666
205.725263609097
216.3293170071019
203.1547681029956
205.27239286450822
200.25016007830752
194.4166692672583
191.2633105783036
193.7234529583573
189.85964094489435
191.22423695046362
187.29088238764987
187.1827581183305
186.87112739321353
192.78039721048086
190.37534846782194
186.85532098925876
191.53725668466598
185.9700777064961
188.10250613190087
186.1800610601977
186.2516255258683
185.8124036248785
186.48370534331008
185.92434159917644
186.13182410843365
185.8140973401657
185.8626824569369
185.82758933105964
185.83938416170716
185.822764962509
185.80309183859382
185.83938807724726
185.8526379724304
185.82529002933586
185.81291627511717
185.6883868881643
185.83152121382622


/var/folders/38/hhm6fff54vb3z3xq7k525lpm0000gn/T/ipykernel_4843/4173131235.py:35: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = scipy.optimize.fmin(func = costfun,x0=[1,0.8])


nan
9
380.55258814626865
402.7198787236984
399.27911202496045
376.22350812011706
362.013971006017
344.0450570026344
316.41871590674555
293.5043083012241
242.6149484357048
199.81145460797438
119.25142824789927
67.90510713668901
157.93411539934226
84.19173472573812
77.34003523315562
106.12253985221926
65.74550135323278
145.69855191849229
67.94166670025362
82.70990744629486
65.94325517597966
62.83792819698516
60.58118063412119
61.09128515323976
55.97551315838689
52.865549857773395
55.229866767019686
54.160499286744276
53.8777726517858
53.531141220584516
61.09128517854313
51.514659500400796
52.387484890348965
53.729731380607404
51.949663089465716
52.48958500158038
51.78933624551945
51.84427536601617
51.57851815916116
52.237186226603185
51.54286514582881
51.52170526862574
51.73647811057212
51.49339271875334
51.579082157062544
51.504731631721896
51.55999749249967
51.51112194514824
51.50235348914295
51.542090254545045
51.50079823160375
51.50155883535034
51.49827192549136
51.502699646439495
51

In [ ]:
 import matplotlib.pyplot as plt
    #plt.plot(t1,y1[0,:])
    #plt.plot(t1,y1[1,:],linewidth=0.5)
    #plt.plot(t1,y1[2,:],linewidth=0.5)  #This plot doesn't match the original
    #plt.plot(t1,y1[3,:],linewidth=0.5) #This plot doesn't match the original

    #sol = solve_ivp(odefun,tspan,init)
    #[t,y] = [sol.t,sol.y]

    #[y,t] = [sol[:,0], sol[:,1]]

    #print("Y value is",sol.y[:,-1])
    #print("Sol.T is",sol.t.shape)

   